获取RS、HB、BO的精度， 其中 baseline 是指没有使用剪枝时对应的精度， 即n_high_performing_model==14时的精度

我只优化100次。

这里的*剪枝*指先跳出算法

In [73]:
import sys

sys.path.append("/Users/sunwu/SW-Research/AutoML-Benchmark")
sys.path.append("/Users/sunwu/SW-Research/AutoML-Benchmark/deps")
from tshpo.lib_class import AnaHelper
import pandas as pd
from scipy.stats import stats
from tshpo.lib_class import FileHelper

EPOCH_KEY = "metric_100"
# EPOCH_KEY = "metric_400"
files = [
    "c04_tshpo_acc_compare_v11_original_20241101_1810.csv.gz",
    "c04_tshpo_acc_compare_v12_middle_original_20241103_0821.csv.gz",
    "c04_tshpo_acc_compare_v12_big_original_20241102_1631.csv.gz",
]
t_df = []
for _f in files:
    # t_df.append(AnaHelper.load_exp_from_csv_tshpo(_f, iterations=[200, 300, 400, 500]))
    t_df.append(AnaHelper.load_exp_from_csv_tshpo(_f, iterations=[100]))
SELECT_TOP_N = 2
process_df = pd.concat(t_df)
process_df = AnaHelper.append_data_size_type(process_df)
process_df = AnaHelper.append_normal_hpo_name(process_df)

outputs = []
# 选择2、3、4个算法

# for _n_high_perf_model in [2, 3, 4]:
for _n_high_perf_model in [SELECT_TOP_N]:
    _df = process_df[
        (process_df['n_high_performing_model'].isin([_n_high_perf_model, 14])) & (process_df['metric'] != "f1")]

    _cdf = pd.pivot_table(_df, index=['metric', 'hpo_opt_method', 'data_size_type', 'dataset', 'fold_index'],
                          columns=['n_high_performing_model'],
                          values=[EPOCH_KEY]).reset_index()
    # print(_cdf.columns)
    # baseline 对应选择所有的14个算法，即不剪枝
    _cdf.columns = ['metric', 'hpo_opt_method', 'data_size_type', 'dataset', 'fold_index', "tshpo", "baseline"]
    _cdf['improve'] = _cdf['tshpo'] - _cdf['baseline']
    _cdf['n_top'] = _n_high_perf_model
    _cdf = _cdf.round(4)

    outputs.append(_cdf)
mdf = pd.concat(outputs, axis=0)
mdf

,metric,hpo_opt_method,data_size_type,dataset,fold_index,tshpo,baseline,improve,n_top
0,accuracy,BO,large,PhishingWebsites,0,0.9711,0.9494,0.0217,2
1,accuracy,BO,large,PhishingWebsites,1,0.9729,0.9584,0.0145,2
2,accuracy,BO,large,PhishingWebsites,2,0.9765,0.9602,0.0163,2
3,accuracy,BO,large,PhishingWebsites,3,0.9801,0.9611,0.0190,2
4,accuracy,BO,large,PhishingWebsites,4,0.9819,0.9530,0.0289,2
...,...,...,...,...,...,...,...,...,...
835,roc_auc,RS,small,tic-tac-toe,0,1.0000,1.0000,0.0000,2
836,roc_auc,RS,small,tic-tac-toe,1,1.0000,1.0000,0.0000,2
837,roc_auc,RS,small,tic-tac-toe,2,1.0000,1.0000,0.0000,2
838,roc_auc,RS,small,tic-tac-toe,3,1.0000,1.0000,0.0000,2


In [74]:
import numpy as np


def myfun(x):
    return x


DECIMAL = 3


## 获取不剪枝情况下BO、RS、HB的性能
def get_method_perf(select_opt_method, column_name):
    """ 
    select_opt_method: one of RS, HB, BO
    column_name: one of tshpo, baseline. tshpo 对应剪枝的结果， baseline对应不剪枝的结果
    """

    ana_data = mdf.query(f"hpo_opt_method=='{select_opt_method}'")
    _tf = ana_data.groupby(by=['dataset', 'metric'])[column_name].agg(['mean', 'std']).reset_index()

    _tf['value'] = _tf.apply(lambda x: f"{np.round(x['mean'], 3)}({np.round(x['std'], DECIMAL)})", axis=1)

    # return _tf
    _tf = pd.pivot_table(_tf, index="dataset", columns=['metric'], values='value', aggfunc=myfun)
    _tf = _tf.reset_index()
    _tf.columns = ["dataset", (select_opt_method, "Accuracy"), (select_opt_method, "ROC AUC")]
    res_file = f"/Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/{select_opt_method}.csv"
    print(f"saved to {res_file}")
    _tf.to_csv(res_file, index=False)


get_method_perf("RS", "baseline")


saved to /Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/RS.csv


In [75]:
get_method_perf("RS", "baseline")
get_method_perf("BO", "baseline")
get_method_perf("HB", "baseline")

saved to /Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/RS.csv
saved to /Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/BO.csv
saved to /Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/HB.csv


In [76]:
# 获取TS-CASH的性能， 也就是兼职后的性能
select_opt_method = "BO"
column_name = "tshpo"
ana_data = mdf.query(f"hpo_opt_method=='{select_opt_method}'")
_tf = ana_data.groupby(by=['dataset', 'metric'])[column_name].agg(['mean', 'std']).reset_index()

_tf['value'] = _tf.apply(lambda x: f"{np.round(x['mean'], 3)}({np.round(x['std'], DECIMAL)})", axis=1)

# return _tf
_tf = pd.pivot_table(_tf, index="dataset", columns=['metric'], values='value', aggfunc=myfun)
_tf = _tf.reset_index()
_tf.columns = ["dataset", ("TS-CASH", "Accuracy"), ("TS-CASH", "ROC AUC")]
res_file = f"/Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/TS-CASH.csv"
print(f"saved to {res_file}")
_tf.to_csv(res_file, index=False)


saved to /Users/sunwu/SW-Research/AutoML-Benchmark/sota/results/TS-CASH.csv
